In [ ]:
import sklearn as sl
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

csv_file = 'https://raw.githubusercontent.com/nimamahmoudi/faas-keep-alive-modeling/master/datasets/wc98/invocation_count.csv'
df = pd.read_csv(csv_file, index_col=0, parse_dates=True)
f = plt.figure(figsize=(10,12))
df['1998-06-30 08:00:01':'1998-07-01 08:00:00'].plot()
users = df['1998-06-30 08:00:01':'1998-07-01 08:00:00'].values
plt.subplot(211)
plt.plot(users)
scaling = sl.preprocessing.MinMaxScaler()
scaled_users = np.ceil(120*scaling.fit_transform(users)).astype(int)
plt.subplot(212)
plt.plot(scaled_users)
plt.show(block = True)

In [ ]:
import subprocess
import time
import sys
import time
import math
from kubernetes import client, config
import os

output = subprocess.check_output('kubectl get pods', shell=True)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/ubuntu/jupyter/google.json"
dir_path = os.path.dirname(os.path.realpath('__file__'))
config.load_kube_config()
DEPLOYMENT_NAME = 'wordpress'
DEPLOYMENT_NS = 'default'


api_instance = client.ExtensionsV1beta1Api()

def get_replica_and_ready():
    api_response = api_instance.read_namespaced_deployment(DEPLOYMENT_NAME, DEPLOYMENT_NS)
    return api_response.status.replicas, api_response.status.ready_replicas

def set_replica_num(rnum):
    rnum = int(rnum)
    if rnum < 1:
        rnum = 1
    api_response = api_instance.read_namespaced_deployment(DEPLOYMENT_NAME, DEPLOYMENT_NS)
    api_response.spec.replicas = rnum
    api_instance.patch_namespaced_deployment_scale(DEPLOYMENT_NAME, DEPLOYMENT_NS, api_response)
u_pre = 6

print(get_replica_and_ready())
set_replica_num(u_pre)


In [ ]:
# Control Initialization KP KI KD and Setpoint 
KP = 0.0004
KI = 0.0004
KD = 0.00005
r_setpoint = 800
e = np.zeros([3,1])
e_pre = np.zeros([3,1])
x_PID = np.zeros([3,1])
r = r_setpoint

In [ ]:
#%% Change working directory from the workspace root to the ipynb file location. Turn this addition off with the DataScience.changeDirOnImportExport setting
import os
from tqdm.auto import tqdm
try:
    os.chdir(os.path.join(os.getcwd(), 'examples'))
    print(os.getcwd())
except:
    pass
import pacs_load_tester as load_tester

#%%
def custom_sensing():
    r1,r2 = get_replica_and_ready()
    return {'r1':r1 , 'r2':r2}

lt = load_tester.pacsLoadTester(hatch_rate=1, temp_stat_max_len=5, base='http://35.232.158.172:8089/')
lt.custom_sensing = custom_sensing
lt.custom_sensing()
tqdm.pandas()

loop_timer = load_tester.TimerClass()
total_timer = load_tester.TimerClass()
user_sequence = [30,60,30,60,30]
lt.custom_sensing = custom_sensing
lt.change_count(user_sequence[0])
lt.start_capturing()
loop_time_in_secs = load_tester.get_loop_time_in_secs('20s')
loop_timer.tic()
total_timer.tic()
results = None

for i in tqdm(range(len(user_sequence)*36)):
    print("U(t)  : ", u_pre)
    user_count = user_sequence[math.floor(i/36)]
    lt.change_count(user_count)

    sleep_time = loop_time_in_secs - loop_timer.toc()
    if sleep_time > 0:
        time.sleep(sleep_time)

    loop_timer.tic()

    result = lt.get_all_stats()
    df_result = pd.DataFrame(data=result)

    if results is None:
        results = df_result
    else:
        results = results.append(df_result)

    #####Identification
    if (result['current_response_time_average'])!=[]:
        if result['current_response_time_average'][-1]!=None :
            e_pre[0] = e[0]
            e_pre[1] = e[1]
            e_pre[2] = e[2]
            e[0]= -(r - np.nanmean(np.array(result['current_response_time_average'],dtype=float)))
            current_response = np.nanmean(np.array(result['current_response_time_average'],dtype=float))
            if e[0]>=-300 and e[0]<=0:
                e[0] = 0
            e[1]= e_pre[0]
            e[2]= e_pre[1]
            x_PID[0] = e[0] - e[1]
            x_PID[1] = e[0]  
            x_PID[2] = e[0] - 2*e[1] + e[2]
            print("Error 0 : ",e[0], "Error 1 : ", e[1],"Error 2 : ", e[2])
            if u_pre ==40 or u_pre <= 1:
                x_PID[1] = 0    
            u_pre =u_pre+ KP*x_PID[0] + KI*x_PID[1] + KD*x_PID[2]
            try :
                set_replica_num(np.ceil(u_pre))
            except :
                print("exception occured")
            
            print("Number of Replica Request: ",result['custom_r1'][-1],"Number of Replica Ready: ",result['custom_r2'][-1],"control signal : ",u_pre,"error :", r_setpoint-current_response)
            print("x1 : ",x_PID[0],"x2 : ",x_PID[1], "x3 : ",x_PID[2], "Tracking error : ",e[0])
            print("Current Response Time is : " , current_response)
lt.stop_test()

results, filename = lt.prepare_results_from_df(results)

results.head()

In [ ]:
res = results
res = res[res['total_rps'] > 0]
responsetime_avg = np.nanmean(res['current_response_time_average'])
responsetime_var = np.sqrt(np.var(res['current_response_time_average']))
container_avg = np.average(res['custom_r2'])
container_var = np.sqrt(np.var(res['custom_r2']))
print("average res : ", responsetime_avg , "Var Res : ",responsetime_var, "average Container : ",container_avg, "Var Container : ", container_var )
import matplotlib.pyplot as plt

get_ipython().run_line_magic('matplotlib', 'inline')

plt.figure(figsize=(12,18),dpi=200)
plt.subplot(511)
#plt.plot(res['elapsed_min'], res['min_response_time'], label='min_response_time')
#plt.plot(res['elapsed_min'], res['current_response_time_percentile_50'], label='median_response_time')
plt.plot(res['elapsed_min'], res['current_response_time_average'], label='avg_response_time')
plt.plot(res['elapsed_min'],r_setpoint*np.ones(len(res['current_response_time_average'])), label='setpoint')
#plt.plot(res['elapsed_min'], res['current_response_time_percentile_95'], label='95th percentile')
#plt.plot(res['elapsed_min'], res['max_response_time'], label='max_response_time')
plt.ylim((0,4000))
plt.xlabel('Time (minutes)')
plt.ylabel('Average Response Time (ms)')
plt.legend()

plt.subplot(512)
plt.plot(res['elapsed_min'], res['user_count'])
plt.xlabel('Time (minutes)')
plt.ylabel('Num of Users')

plt.subplot(513)
plt.plot(res['elapsed_min'], res['total_rps'])
plt.xlabel('Time (minutes)')
plt.ylabel('Throughput (req/s)')

plt.subplot(514)
plt.plot(res['elapsed_min'], res['fail_ratio'])
plt.xlabel('Time (minutes)')
plt.ylabel('Fail Ratio')

plt.subplot(515)
plt.plot(res['elapsed_min'], res['custom_r1'])
plt.plot(res['elapsed_min'], res['custom_r2'])
plt.xlabel('Time (minutes)')
plt.ylabel('Custom Value')



filename = filename.replace('aa.csv', '')
plt.savefig(filename + '.png', dpi=300)
plt.savefig(filename + '.pdf')
plt.show()


